In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Colombia,CO,10.9878,-74.9547,80.69,84,100,7.83,overcast clouds
1,1,Torbay,CA,47.6666,-52.7314,41.88,84,100,8.05,overcast clouds
2,2,Atuona,PF,-9.8000,-139.0333,77.90,69,5,16.04,clear sky
3,3,Aksu,CN,41.1231,80.2644,82.87,17,83,10.58,broken clouds
4,4,Hobart,AU,-42.8794,147.3294,48.25,83,40,8.05,scattered clouds


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                    & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Colombia,CO,10.9878,-74.9547,80.69,84,100,7.83,overcast clouds
2,2,Atuona,PF,-9.8000,-139.0333,77.90,69,5,16.04,clear sky
3,3,Aksu,CN,41.1231,80.2644,82.87,17,83,10.58,broken clouds
8,8,Hukuntsi,BW,-24.0000,21.7500,76.23,32,45,14.83,scattered clouds
11,11,Kavieng,PG,-2.5744,150.7967,82.00,75,100,8.97,overcast clouds
12,12,Port Elizabeth,ZA,-33.9180,25.5701,86.09,48,0,11.50,clear sky
13,13,Rikitea,PF,-23.1203,-134.9692,76.17,77,100,16.69,light rain
14,14,Manono,CD,-7.3000,27.4167,88.97,38,22,6.58,few clouds
20,20,Butaritari,KI,3.0707,172.7902,81.28,76,51,11.36,broken clouds
29,29,Nabire,ID,-3.3667,135.4833,77.31,86,94,3.33,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                242
City                   242
Country                241
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Colombia,CO,80.69,overcast clouds,10.9878,-74.9547,
2,Atuona,PF,77.90,clear sky,-9.8000,-139.0333,
3,Aksu,CN,82.87,broken clouds,41.1231,80.2644,
8,Hukuntsi,BW,76.23,scattered clouds,-24.0000,21.7500,
11,Kavieng,PG,82.00,overcast clouds,-2.5744,150.7967,
12,Port Elizabeth,ZA,86.09,clear sky,-33.9180,25.5701,
13,Rikitea,PF,76.17,light rain,-23.1203,-134.9692,
14,Manono,CD,88.97,few clouds,-7.3000,27.4167,
20,Butaritari,KI,81.28,broken clouds,3.0707,172.7902,
29,Nabire,ID,77.31,overcast clouds,-3.3667,135.4833,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping')

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.count()

City                   228
Country                228
Max Temp               228
Current Description    228
Lat                    228
Lng                    228
Hotel Name             228
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))